In [4]:
from openbabel import openbabel

In [5]:
from rdkit import Chem
from rdkit.Chem import rdBase
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Descriptors
from rdkit.Chem import Crippen

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as ss

In [7]:
#matches_data = pd.read_csv('./MatchedDataNoStereo/FormoseAmm/NoStereoFormoseAmm.tsv', sep='\t')

In [8]:
#FormoseAmm_rels = pd.read_csv('./FormoseAmmData/FormoseAmm_rels.tsv', sep='\t')

In [126]:
def split(word):
    return[char for char in word]

In [127]:
def index_finder(Smiles, pathway, rels_data):
    index = 0
    rule = 0
    for i in range(len(rels_data['Reagent'])):
        if rels_data['Reagent'][i] == Smiles and rels_data['Formed/Produced'][i] == 1 and precursor_finder(rels_data['Index'][i], rels_data) not in pathway and precursor_finder(rels_data['Index'][i], rels_data) != rels_data['Reagent'][i]:
            index = rels_data['Index'][i] 
            rule = rels_data['Rule'][i]
            break
    return(index, rule)

In [128]:
def precursor_finder(index, rels_data):
    precursor = 0
    for i in range(len(rels_data['Index'])):
        if rels_data['Index'][i] == index and rels_data['Formed/Produced'][i] == -1:
            precursor = rels_data['Reagent'][i]
            break
    return(precursor)

In [129]:
def pathway_finder(Smiles, rels_data):
    pathway = [Smiles]
    G0 = ['N', 'C=O', 'C(CO)=O', 'O']
    #G0 = ['C=O', 'C(CO)=O']
    pathway_complete = False
    while pathway_complete == False:
        index = 0
        rule = 0
        index, rule = index_finder(pathway[-1], pathway, rels_data)
        if index == 0 or pathway[-1] in G0:
            pathway_complete == True
            break
        else:
            precursor = precursor_finder(index, rels_data)
            precursors = []
            products = []
            for i in range(len(rels_data['Index'])):
                if rels_data['Index'][i] == index and rels_data['Formed/Produced'][i] == -1:
                    precursors.append(rels_data['Reagent'][i])
                elif rels_data['Index'][i] == index and rels_data['Formed/Produced'][i] == 1:
                    products.append(rels_data['Reagent'][i])
            reagents = ''
            if len(precursors) == 0:
                reagents += f'{precursors}[0]'
            else:
                for i in range(len(precursors)-1):
                    reagents += f'{precursors[i]} + '
                reagents += f'{precursors[-1]}'
            produced = ''
            if len(products) == 0:
                produced += f'{products}[0]'
            else:
                for i in range(len(products)-1):
                    produced += f'{products[i]} + '
                produced += f'{products[-1]}'
            pathway.append([index, rule, reagents, '->', produced])
            pathway.append(precursor)
    return(pathway)

In [130]:
def pathways(matches, rels):
    matches_data = pd.read_csv(matches, sep='\t')
    rels_data = pd.read_csv(rels, sep='\t')
    pathways = []
    for i in range(1): #len(matches_data['Smiles'])
        pathways.append((pathway_finder(matches_data['Smiles'][i], rels_data)))
        print(i)
    #G0 = ['N', 'C=O', 'C(CO)=O', 'O']
    G0 = ['C=O', 'C(CO)=O']
    counter = 0
    for i in range(len(pathways)): 
        if pathways[i][-1] not in G0:
            counter +=1
    print(f'Errors = {counter}')
    print(f'No. matches = {len(pathways)}')
    #matches_data['Pathways'] = pathways
    return(pathways)

In [131]:
%%time
a = pathways('./MatchedDataNoStereo/PyruvicAcid/NoStereoPyruvicAcid_CHNO.tsv', './PyruvicAcidData/PyruvicAcid_rels.tsv')
#df.to_csv(r'PyruvicAcidPathways.tsv', header=None, index=None, sep='\t', mode='a') 

0
Errors = 1
No. matches = 1
CPU times: user 9.53 s, sys: 83.9 ms, total: 9.61 s
Wall time: 9.8 s


In [141]:
a[0][5]

['150_0',
 'Benzilic Acid Rearrangement',
 'O + C(C(O)=O)(CC(C(C)=O)=O)=O',
 '->',
 'C(C(C)(CC(C(O)=O)=O)O)(O)=O']